In [1]:
# Some imports, we are not gong to use all the imports in this workbook but in subsequent workbooks we surely will.
import os
import time

import numpy as np
import pandas as pd

from toolz import pipe as p
import tensorflow as tf


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

In [2]:
import model_runs
import models as models
import utils as u

from importlib import reload
reload(u)
reload(model_runs)
reload(models)

Using TensorFlow backend.


<module 'models' from 'C:\\Users\\jod204\\projects\\jo\\samplesize_test\\models.py'>

# Settings and Helpers

In [3]:
# Define some Global Variables
max_features = 20000 # Maximum Number of words we want to include in our dictionary
maxlen = 400 # No of words in question we want to create a sequence with
embed_size = 50# Size of word to vec embedding we are using


def get_log_dir(iteration, samples_per_class):
    return f"output/{samples_per_class}/{iteration}/"


def save_series_gen(log_dir, prefix):
    def save_series(series, name):
        series.to_csv(f'{log_dir}/{prefix}_{name}.csv', index = False, header = False)
    
    return save_series


def log_truths(log_dir, train_y, val_y, test_y):
    save_series = save_series_gen(log_dir, 'true')
        
    save_series(train_y, 'train')
    save_series(val_y, 'val')
    save_series(test_y, 'test')


def log_iteration_preds(log_dir, model_name,
                  pred_train_y, pred_val_y, pred_test_y,
                  one_train_time):
    def save_arr(arr, name):
        arr.tofile(f'{log_dir}/{model_name}_{name}.csv', sep = ',')
    
    save_arr(pred_train_y, 'pred_train')
    save_arr(pred_val_y, 'pred_val')
    save_arr(pred_test_y, 'pred_test')
    p(one_train_time,
      np.array,
      lambda _: save_arr(_, 'time'))


def run_and_log_model(model, model_name, train_X, train_y, val_X, val_y, test_X, iteration, samples_per_class):
    start_time = time.perf_counter()
    pred_train_y, pred_val_y, pred_test_y = model_runs.train_pred(model, train_X, train_y,
                                          val_X, val_y, test_X, epochs=50)
    end_time = time.perf_counter()
    one_train_time = end_time - start_time
    
    dir_name = get_log_dir(iteration, samples_per_class)
    log_iteration_preds(dir_name, model_name, 
                  pred_train_y, pred_val_y, pred_test_y,
                  one_train_time) 


# Model Execution

In [5]:
data = pd.read_csv('./content/pubmed_cr_hep_ctl_abstracts_clean.csv')

for samples_per_class in range(500, 2500, 500):
  for i in range(0, 25):
      train_X, val_X, test_X, train_y, val_y, test_y, word_index = u.load_and_prec(
          samples_per_class, max_features, maxlen = maxlen, data = data)

      log_dir = get_log_dir(i, samples_per_class)
      os.makedirs(log_dir, exist_ok = True)

      log_truths(log_dir, train_y, val_y, test_y)

      print(test_X.shape[1])
      sequence_len = test_X.shape[1]

      def run_and_log_model_curry(model, model_name):
        run_and_log_model(model, model_name, 
                          train_X, train_y, val_X, val_y, test_X, 
                          i, samples_per_class)

      # CNN
      cnn_model_built = models.cnn_model(sequence_len, [3, 4, 5], maxlen, max_features, embed_size, 
                                         num_filters=200, drop_rate=0.2)
      cnn_model_built.summary()
      run_and_log_model_curry(cnn_model_built, 'cnn')

      # LSTM
      lstm_model = models.model_lstm_du(maxlen, max_features, embed_size)
      lstm_model.summary()
      run_and_log_model_curry(lstm_model, 'lstm')

      # HAN
      han_model = models.model_lstm_atten(maxlen, max_features, embed_size)
      han_model.summary()
      run_and_log_model_curry(han_model, 'han')


1000
500
500
X shape :  (1000,)
y shape :  (1000,)
400
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 400, 50)      1000000     input_10[0][0]                   
__________________________________________________________________________________________________
reshape_7 (Reshape)             (None, 400, 50, 1)   0           embedding_10[0][0]               
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 398, 1, 200)  30200       reshape_7[0][0]                  
______________________________________________________

603/603 [==============================] - 0s 192us/step - loss: 0.2081 - acc: 0.9801 - val_loss: 0.2975 - val_acc: 0.9701
Epoch 36/50
603/603 [==============================] - 0s 194us/step - loss: 0.2020 - acc: 0.9818 - val_loss: 0.2935 - val_acc: 0.9701
Epoch 37/50
603/603 [==============================] - 0s 192us/step - loss: 0.1959 - acc: 0.9818 - val_loss: 0.2882 - val_acc: 0.9701
Epoch 38/50
603/603 [==============================] - 0s 192us/step - loss: 0.1879 - acc: 0.9834 - val_loss: 0.2821 - val_acc: 0.9701
Epoch 39/50
603/603 [==============================] - 0s 168us/step - loss: 0.1816 - acc: 0.9834 - val_loss: 0.2780 - val_acc: 0.9701
Epoch 40/50
603/603 [==============================] - 0s 190us/step - loss: 0.1776 - acc: 0.9834 - val_loss: 0.2740 - val_acc: 0.9701
Epoch 41/50
603/603 [==============================] - 0s 177us/step - loss: 0.1721 - acc: 0.9834 - val_loss: 0.2686 - val_acc: 0.9701
Epoch 42/50
603/603 [==============================] - 0s 192us/ste

Epoch 25/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6814 - acc: 0.5191 - val_loss: 0.6920 - val_acc: 0.4478
Epoch 26/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6810 - acc: 0.5224 - val_loss: 0.6918 - val_acc: 0.4478
Epoch 27/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6803 - acc: 0.5224 - val_loss: 0.6914 - val_acc: 0.4478
Epoch 28/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6801 - acc: 0.5290 - val_loss: 0.6910 - val_acc: 0.4478
Epoch 29/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6796 - acc: 0.5340 - val_loss: 0.6906 - val_acc: 0.4478
Epoch 30/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6790 - acc: 0.5423 - val_loss: 0.6901 - val_acc: 0.4478
Epoch 31/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6782 - acc: 0.5439 - val_loss: 0.6897 - val_acc: 0.4478
Epoch 32/50
603/603 [==============================] - 2s 3ms/step - 

Train on 603 samples, validate on 67 samples
Epoch 1/50
603/603 [==============================] - 4s 6ms/step - loss: 0.7509 - acc: 0.5174 - val_loss: 0.7301 - val_acc: 0.4925
Epoch 2/50
603/603 [==============================] - 0s 182us/step - loss: 0.7269 - acc: 0.5108 - val_loss: 0.7141 - val_acc: 0.4925
Epoch 3/50
603/603 [==============================] - 0s 192us/step - loss: 0.7112 - acc: 0.7264 - val_loss: 0.7030 - val_acc: 0.8507
Epoch 4/50
603/603 [==============================] - 0s 192us/step - loss: 0.7001 - acc: 0.9055 - val_loss: 0.6947 - val_acc: 0.8657
Epoch 5/50
603/603 [==============================] - 0s 192us/step - loss: 0.6912 - acc: 0.9585 - val_loss: 0.6875 - val_acc: 0.8657
Epoch 6/50
603/603 [==============================] - 0s 192us/step - loss: 0.6828 - acc: 0.9569 - val_loss: 0.6800 - val_acc: 0.8209
Epoch 7/50
603/603 [==============================] - 0s 192us/step - loss: 0.6734 - acc: 0.9237 - val_loss: 0.6716 - val_acc: 0.8209
Epoch 8/50
603/603 

Train on 603 samples, validate on 67 samples
Epoch 1/50
603/603 [==============================] - 7s 11ms/step - loss: 0.6932 - acc: 0.5075 - val_loss: 0.6929 - val_acc: 0.4925
Epoch 2/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6916 - acc: 0.5191 - val_loss: 0.6920 - val_acc: 0.5522
Epoch 3/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6902 - acc: 0.6700 - val_loss: 0.6912 - val_acc: 0.5373
Epoch 4/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6891 - acc: 0.5390 - val_loss: 0.6904 - val_acc: 0.5373
Epoch 5/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6871 - acc: 0.5771 - val_loss: 0.6895 - val_acc: 0.6567
Epoch 6/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6843 - acc: 0.8292 - val_loss: 0.6881 - val_acc: 0.7015
Epoch 7/50
603/603 [==============================] - 2s 3ms/step - loss: 0.6809 - acc: 0.8342 - val_loss: 0.6859 - val_acc: 0.7313
Epoch 8/50
603/603 [==========

...

Train on 2412 samples, validate on 268 samples
Epoch 1/50
2412/2412 [==============================] - 670s 278ms/step - loss: 0.6934 - acc: 0.4975 - val_loss: 0.6915 - val_acc: 0.4888
Epoch 2/50
2412/2412 [==============================] - 9s 4ms/step - loss: 0.6898 - acc: 0.6070 - val_loss: 0.6879 - val_acc: 0.5410
Epoch 3/50
2412/2412 [==============================] - 9s 4ms/step - loss: 0.6848 - acc: 0.6837 - val_loss: 0.6808 - val_acc: 0.8433
Epoch 4/50
2412/2412 [==============================] - 9s 4ms/step - loss: 0.6725 - acc: 0.8520 - val_loss: 0.6603 - val_acc: 0.8619
Epoch 5/50
2412/2412 [==============================] - 9s 4ms/step - loss: 0.6310 - acc: 0.9051 - val_loss: 0.5522 - val_acc: 0.8507
Epoch 6/50
2412/2412 [==============================] - 9s 4ms/step - loss: 0.4302 - acc: 0.8943 - val_loss: 0.5784 - val_acc: 0.7575

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/50
2412/2412 [==============================] - 9s 4ms/s

Train on 2412 samples, validate on 268 samples
Epoch 1/50
2412/2412 [==============================] - 669s 277ms/step - loss: 0.7332 - acc: 0.4764 - val_loss: 0.7064 - val_acc: 0.4813
Epoch 2/50
2412/2412 [==============================] - 1s 375us/step - loss: 0.6955 - acc: 0.5004 - val_loss: 0.6818 - val_acc: 0.8246
Epoch 3/50
2412/2412 [==============================] - 1s 361us/step - loss: 0.6710 - acc: 0.9100 - val_loss: 0.6546 - val_acc: 0.9440
Epoch 4/50
2412/2412 [==============================] - 1s 361us/step - loss: 0.6366 - acc: 0.9395 - val_loss: 0.6145 - val_acc: 0.9440
Epoch 5/50
2412/2412 [==============================] - 1s 362us/step - loss: 0.5873 - acc: 0.9449 - val_loss: 0.5642 - val_acc: 0.9478
Epoch 6/50
2412/2412 [==============================] - 1s 361us/step - loss: 0.5290 - acc: 0.9428 - val_loss: 0.5094 - val_acc: 0.9515
Epoch 7/50
2412/2412 [==============================] - 1s 359us/step - loss: 0.4693 - acc: 0.9424 - val_loss: 0.4592 - val_acc: 0.9478

2412/2412 [==============================] - 9s 4ms/step - loss: 0.6895 - acc: 0.5220 - val_loss: 0.6864 - val_acc: 0.5970
Epoch 3/50
2412/2412 [==============================] - 9s 4ms/step - loss: 0.6822 - acc: 0.6153 - val_loss: 0.6766 - val_acc: 0.7239
Epoch 4/50
2412/2412 [==============================] - 9s 4ms/step - loss: 0.6645 - acc: 0.7624 - val_loss: 0.6439 - val_acc: 0.7612
Epoch 5/50
2412/2412 [==============================] - 9s 4ms/step - loss: 0.5937 - acc: 0.8354 - val_loss: 0.4635 - val_acc: 0.8396
Epoch 6/50
2412/2412 [==============================] - 9s 4ms/step - loss: 0.3714 - acc: 0.8827 - val_loss: 0.4781 - val_acc: 0.8134

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/50
2412/2412 [==============================] - 9s 4ms/step - loss: 0.3430 - acc: 0.8694 - val_loss: 0.4938 - val_acc: 0.8097

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 8/50
2412/2412 [=======================

Train on 2412 samples, validate on 268 samples
Epoch 1/50
2412/2412 [==============================] - 713s 296ms/step - loss: 0.7317 - acc: 0.5701 - val_loss: 0.7064 - val_acc: 0.4403
Epoch 2/50
2412/2412 [==============================] - 1s 379us/step - loss: 0.6945 - acc: 0.5058 - val_loss: 0.6840 - val_acc: 0.4403
Epoch 3/50
2412/2412 [==============================] - 1s 368us/step - loss: 0.6671 - acc: 0.5709 - val_loss: 0.6513 - val_acc: 0.8806
Epoch 4/50
2412/2412 [==============================] - 1s 374us/step - loss: 0.6289 - acc: 0.9229 - val_loss: 0.6076 - val_acc: 0.9478
Epoch 5/50
2412/2412 [==============================] - 1s 374us/step - loss: 0.5766 - acc: 0.9465 - val_loss: 0.5544 - val_acc: 0.9590
Epoch 6/50
2412/2412 [==============================] - 1s 377us/step - loss: 0.5191 - acc: 0.9502 - val_loss: 0.4997 - val_acc: 0.9627
Epoch 7/50
2412/2412 [==============================] - 1s 361us/step - loss: 0.4624 - acc: 0.9469 - val_loss: 0.4478 - val_acc: 0.9552

Train on 2412 samples, validate on 268 samples
Epoch 1/50
2412/2412 [==============================] - 723s 300ms/step - loss: 0.6929 - acc: 0.4983 - val_loss: 0.6935 - val_acc: 0.4403
Epoch 2/50
2412/2412 [==============================] - 8s 3ms/step - loss: 0.6893 - acc: 0.5493 - val_loss: 0.6903 - val_acc: 0.5187
Epoch 3/50
2412/2412 [==============================] - 8s 3ms/step - loss: 0.6837 - acc: 0.6368 - val_loss: 0.6820 - val_acc: 0.7090
Epoch 4/50
2412/2412 [==============================] - 8s 3ms/step - loss: 0.6695 - acc: 0.8105 - val_loss: 0.6583 - val_acc: 0.8097
Epoch 5/50
2412/2412 [==============================] - 8s 3ms/step - loss: 0.6265 - acc: 0.8773 - val_loss: 0.5782 - val_acc: 0.8358
Epoch 6/50
2412/2412 [==============================] - 8s 3ms/step - loss: 0.4520 - acc: 0.9017 - val_loss: 0.4396 - val_acc: 0.8396
Epoch 7/50
2412/2412 [==============================] - 8s 3ms/step - loss: 0.3633 - acc: 0.8818 - val_loss: 0.5243 - val_acc: 0.8060

Epoch 0000